In [1]:
import os
import sys
import numpy as np
import pandas as pd 
import logging
import matplotlib 

from ppp.splines import InoueBspline
from ppp.interpolation import PoissonProcess
from matplotlib import pyplot as plt



matplotlib.rcParams['ps.useafm'] = True
matplotlib.rcParams['pdf.use14corefonts'] = True
matplotlib.rcParams['text.usetex'] = True 

%matplotlib notebook

In [2]:
import pymongo
from pymongo import MongoClient

client = MongoClient("129.26.78.131:27017")

In [3]:
logging.basicConfig(level=logging.INFO)

In [4]:
client.database_names()
db = client["bitcoin"]

#BLOCKS = [block for block in db['blocks_1'].find().limit(100)]
#db["test_blocks"].insert(BLOCKS)

# Aggregation

In [ ]:
for i in range(12, 13):
    db["blocks_{0}".format(i)].aggregate([{"$project":
                                      {"block_index":"$block_index",
                                       "_id":0,
                                       "height":"$block_height",
                                       "relayed_by":"$relayed_by",
                                       "hash": "$hash",
                                       "tx":"$tx"}},
                                 {"$unwind":"$tx"},
                                 {"$project":
                                         {"inputs":"$tx.inputs",
                                          "out":"$tx.out",
                                          "time":"$tx.time",
                                          "hash": "$hash",
                                          "_id":0
                                         }},
                                 {"$unwind":"$inputs"},
                                 {"$project":
                                            {"time":"$time",
                                             "hash": "$hash",
                                             "address_source":"$inputs.prev_out.addr",
                                             "amount":"$inputs.prev_out.value",
                                             "out":"$out"}},
                                 {"$unwind":"$out"},
                                 {"$project":
                                            {"t":"$time",
                                             "tx": "$hash",
                                             "s":"$address_source",
                                             "a_s":"$amount",
                                             "r":"$out.addr",
                                             "tag":"$out.addr_tag_link",
                                             "a_r":"$out.value"}},
                                 {"$out":"blocks{0}_aggregation".format(i)}],allowDiskUse=True)

In [ ]:
for a in db["blocks2_aggregation"].find().limit(10):
    print a
    print "\n"

# Address Collection

In [ ]:
for i in range(1, 13):
    print i
    db["blocks{0}_aggregation".format(i)].aggregate([{"$group":{
                                       "_id":{"coll":"{0}".format(i), "add":'$s'},
                                       "tx":{"$addToSet":"$t"}}},                                    
                                       {"$out":"senders{0}".format(i)}],allowDiskUse=True)

In [ ]:
db["senders"].aggregate([{"$unwind": "$tx"},
                                {"$group": { 
                                            "_id": "$_id.add",                                               
                                            "tx": {"$addToSet":"$tx"},
                                            "ntx": {"$sum": 1}
                                            }
                                },
                              {"$out":"senders_aggregated"}
                             ],
                        allowDiskUse=True)

## Data Preprocessing for infering poisson intensities

In [6]:
tt = db["senders_aggregated"].aggregate([
                                 {"$match": { "ntx": {"$gte": 10}}},
                                 {"$unwind": "$tx"},
                                 {"$match": { "tx": {"$gte": 1483228800}}},
                                 {"$project": { 
                                                "_id": "$_id",                                               
                                                "tx": {"$divide": [{"$subtract": ["$tx", 1483228800]}, 24*60*60.]}  
                                               }
                                 },
                                 {"$group": { 
                                                "_id": "$_id",                                               
                                                "tx": {"$addToSet":"$tx"},
                                                "ntx": {"$sum": 1}
                                               }
                                 },
                                 {"$out":"senders_limit"}
                             ],
                            allowDiskUse=True)

In [28]:
db.getCollection('senders_limit').createIndex({"ntx": 1});

TypeError: 'Collection' object is not callable. If you meant to call the 'createIndex' method on a 'Collection' object it is failing because no such method exists.

In [8]:
for a in db["senders_limit"].find().limit(10):
    print (a)
    print ("\n")
    print ("\n")

{'_id': None, 'tx': [62.37020833333333, 86.73275462962962, 104.05341435185186, 42.36494212962963, 35.501666666666665, 146.9434375, 72.5980324074074, 106.7238888888889, 33.350462962962965, 47.28056712962963, 16.264155092592592, 152.46980324074073, 151.76003472222223, 132.1095138888889, 50.120381944444446, 67.13194444444444, 15.760011574074074, 92.2376736111111, 36.15366898148148, 159.6790625, 100.21453703703703, 141.33119212962964, 45.13925925925926, 126.42407407407407, 95.03297453703703, 149.51820601851853, 86.7938425925926, 81.07532407407408, 151.79033564814816, 7.963715277777778, 105.77547453703704, 147.92152777777778, 59.267986111111114, 137.35136574074073, 162.51413194444444, 159.59930555555556, 105.09498842592592, 67.69188657407408, 145.1588425925926, 132.8287962962963, 161.41736111111112, 28.89591435185185, 46.925300925925924, 7.75037037037037, 80.86333333333333, 64.19365740740741, 162.39253472222222, 37.15511574074074, 74.17546296296297, 4.955150462962963, 134.36005787037038, 15

## Histogram of arrivals

In [ ]:
A = [a["ntx"] for a in db["send"].find()]

In [ ]:
stuff = plt.hist(A[:],bins=10)
plt.loglog()

In [29]:
data = []
addresses = db["senders_limit"].find({}, {"_id": 1, "tx": 1}).sort("ntx")
for address in addresses:
    data.append((address["_id"], address["tx"]))

In [ ]:
b_spline_funcion = InoueBspline(M=12, max_arrival=163., X=np.asarray(data[1]))
b_interpolation = PoissonProcess(b_spline_funcion)
b_interpolation.fit(20, 0.01)
b_interpolation.plot()

In [20]:
addresses = db["senders_limit"].find({}).sort("ntx")

In [39]:
data[400000]

('39dUbDyFiQQjAGWdZ4CnqtMYwG3VrWTr6X',
 [131.0317824074074,
  123.86125,
  113.16520833333334,
  131.89582175925926,
  135.0412037037037,
  138.76097222222222,
  125.9696875,
  128.98728009259258,
  123.26013888888889,
  125.20560185185185,
  119.03068287037037,
  132.98236111111112,
  116.97459490740741,
  112.02996527777778,
  130.05991898148147,
  128.03569444444443,
  120.17380787037037,
  136.71641203703703,
  134.0501273148148,
  118.50820601851852,
  114.16335648148149,
  121.13892361111111,
  116.46645833333334,
  127.17721064814815,
  122.06103009259259])